In [68]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt

class Neuron:
    def __init__(self, id):
        self.id = id
        self.outgoing_connections = []

class Connection:
    def __init__(self, in_neuron, out_neuron, weight):
        self.in_neuron = in_neuron
        self.out_neuron = out_neuron
        self.weight = weight
        self.enabled = True

class Network(nn.Module):
    def __init__(self, num_inputs, num_outputs, connections, hidden_neurons):
        super(Network, self).__init__()
        self.num_inputs = num_inputs
        self.num_outputs = num_outputs
        self.neurons = []
        self.connections = []

        for i in range(num_inputs):
            self.neurons.append(Neuron(i))

        for i in range(num_outputs):
            neuron_id = num_inputs + i
            self.neurons.append(Neuron(neuron_id))

        for conn in connections:
            in_neuron = self.neurons[conn.in_neuron]
            out_neuron = self.neurons[conn.out_neuron]
            connection = Connection(in_neuron, out_neuron, conn.weight)
            self.connections.append(connection)
            in_neuron.outgoing_connections.append(connection)

        for _ in range(hidden_neurons):
            neuron_id = len(self.neurons)
            self.neurons.append(Neuron(neuron_id))

        self.linear = nn.Linear(len(self.neurons), num_outputs)

    def forward(self, x):
        for neuron in self.neurons:
            neuron.value = 0.0

        for i in range(self.num_inputs):
            self.neurons[i].value = x[:, i]  # Update to handle batch input

        for connection in self.connections:
            if connection.enabled:
                connection.out_neuron.value += connection.in_neuron.value * connection.weight

        output = np.array([])
        for i in range(self.num_inputs, self.num_inputs + self.num_outputs):
            output = np.append(output, self.neurons[i].value)
        output = torch.Tensor(output)
        output = self.linear(output)  # Apply linear layer
        return torch.sigmoid(output)  # Apply sigmoid activation


class NEAT:
    def __init__(self, num_inputs, num_outputs, population_size, hidden_neurons):
        self.num_inputs = num_inputs
        self.num_outputs = num_outputs
        self.population_size = population_size
        self.hidden_neurons = hidden_neurons
        self.population = []
        self.history = {'neurons': [], 'loss': [], 'accuracy': []}
        
        for _ in range(population_size):
            connections = []
            for i in range(num_inputs):
                for j in range(num_inputs, num_inputs + num_outputs):
                    connections.append(Connection(i, j, np.random.uniform(-1, 1)))
            
            network = Network(num_inputs, num_outputs, connections, hidden_neurons)
            self.population.append(network)
    
    # The remaining code remains the same
    
    def evaluate(self, network, dataloader, criterion):
        total_loss = 0.0
        correct = 0
        total = 0
        
        with torch.no_grad():
            for inputs, labels in dataloader:
                outputs = network(inputs)

                loss = criterion(outputs, labels)
                total_loss += loss.item()
                
                predicted = outputs.round().long()
                correct += (predicted == labels).sum().item()
                total += labels.size(0)
        
        accuracy = correct / total
        return total_loss, accuracy
    
    def train(self, train_loader, test_loader, num_generations, criterion, optimizer):
        for generation in range(num_generations):
            print(f"Generation {generation+1}/{num_generations}")
            for network in self.population:
                network.train()
                loss, accuracy = self.evaluate(network, train_loader, criterion)
                self.history['neurons'].append(len(network.neurons))
                self.history['loss'].append(loss)
                self.history['accuracy'].append(accuracy)
                
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            
            best_network = max(self.population, key=lambda n: self.evaluate(n, test_loader, criterion)[1])
            print(f"Best Accuracy: {self.evaluate(best_network, test_loader, criterion)[1]}")
            
            # Create new population using crossover and mutation
            new_population = []
            for _ in range(self.population_size):
                parent1 = np.random.choice(self.population)
                parent2 = np.random.choice(self.population)
                child = self.crossover(parent1, parent2)
                self.mutate(child)
                new_population.append(child)
            
            self.population = new_population
    
    def crossover(self, parent1, parent2):
        child_connections = []
        for conn1, conn2 in zip(parent1.connections, parent2.connections):
            if np.random.uniform() < 0.5:
                child_connections.append(Connection(conn1.in_neuron.id, conn1.out_neuron.id, conn1.weight))
            else:
                child_connections.append(Connection(conn2.in_neuron.id, conn2.out_neuron.id, conn2.weight))
        
        child_hidden_neurons = max(len(parent1.neurons), len(parent2.neurons)) - self.num_inputs - self.num_outputs
        child = Network(self.num_inputs, self.num_outputs, child_connections, child_hidden_neurons)
        return child
    
    def mutate(self, network):
        for connection in network.connections:
            if np.random.uniform() < 0.1:  # Mutate weight
                connection.weight += np.random.uniform(-0.1, 0.1)
            if np.random.uniform() < 0.01:  # Toggle connection
                connection.enabled = not connection.enabled
    
    def plot_history(self):
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        plt.plot(self.history['neurons'])
        plt.xlabel('Generation')
        plt.ylabel('Number of Neurons')
        plt.title('Number of Neurons in Hidden Layers')
        
        plt.subplot(1, 2, 2)
        plt.plot(self.history['loss'], label='Loss')
        plt.plot(self.history['accuracy'], label='Accuracy')
        plt.xlabel('Generation')
        plt.ylabel('Value')
        plt.title('Loss and Accuracy')
        plt.legend()
        
        plt.tight_layout()
        plt.show()




In [70]:
num_inputs = 8
num_outputs = 1
population_size = 100
hidden_neurons = 10
num_generations = 10

train_data = np.random.randn(100, num_inputs)
train_labels = np.random.randint(0, 2, (100, num_outputs))
test_data = np.random.randn(50, num_inputs)
test_labels = np.random.randint(0, 2, (50, num_outputs))

train_dataset = TensorDataset(torch.Tensor(train_data), torch.Tensor(train_labels))
test_dataset = TensorDataset(torch.Tensor(test_data), torch.Tensor(test_labels))

train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=10, shuffle=False)

neat = NEAT(num_inputs, num_outputs, population_size, hidden_neurons)
criterion = nn.BCELoss()
optimizer = optim.Adam(neat.population[0].parameters(), lr=0.001)

neat.train(train_loader, test_loader, num_generations, criterion, optimizer)
neat.plot_history()


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x10 and 19x1)